### SA on sentences w/ medication
http://textminingonline.com/dive-into-nltk-part-ix-from-text-classification-to-sentiment-analysis

In [3]:
# Trained with nltk movie_reviews

In [4]:
import nltk
from nltk.corpus import movie_reviews
from random import shuffle
from nltk import tokenize
from nltk.tokenize import TweetTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import pandas as pd
import gensim
import nltk
import numbers
from nltk.sentiment.vader import SentimentIntensityAnalyzer

tknzr = TweetTokenizer()

In [5]:
def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features

In [6]:
documents = [(list(movie_reviews.words(fileid)), category) 
for category in movie_reviews.categories()
for fileid in movie_reviews.fileids(category)]

In [7]:
shuffle(documents)

In [8]:
print (documents[0])

(['seen', 'at', 'the', '21st', 'portland', 'international', 'film', 'festival', 'film', 'title', ':', 'supermarket', 'woman', 'director', ':', 'juzo', 'itami', 'country', ':', 'japan', '1997', 'cinematography', ':', 'yonezo', 'maeda', 'music', ':', 'toshiyuki', 'honda', 'cast', ':', 'nobuko', 'miyamoto', ',', 'masahitko', 'tsugawa', ',', 'shiro', 'ito', ',', 'yuji', 'miyake', ',', 'akiko', 'matsumoto', 'super', 'features', ':', 'typical', 'itami', 'film', '.', '.', '.', 'funny', 'and', 'pointed', 'in', 'its', 'own', 'way', '.', 'if', 'you', 'don', "'", 't', 'care', 'how', 'and', 'what', 'the', 'food', 'you', 'buy', 'goes', 'through', 'in', 'your', 'local', 'supermarket', ',', 'then', 'this', 'film', 'is', 'not', 'for', 'you', '.', '.', '.', '.', 'maybe', 'it', 'is', 'better', 'to', 'ignore', 'it', ',', 'as', 'this', 'strong', 'and', 'biting', 'satire', ',', 'as', 'only', 'juzo', 'itami', 'can', 'deliver', ',', 'will', 'give', 'you', 'an', 'insight', 'as', 'to', 'what', 'some', 'foods',

In [9]:
len(documents)

2000

In [10]:
all_words = nltk.FreqDist(w.lower() for w in movie_reviews.words())

In [11]:
# word_features = all_words.keys()[:2000]
word_features = list(all_words.keys())[:2000]

In [12]:
# len(featuresets)

In [13]:
# print (document_features(movie_reviews.words('pos/cv957_8737.txt')))

In [14]:
featuresets = [(document_features(d), c) for (d, c) in documents]

In [15]:
train_set, test_set = featuresets[100:], featuresets[:100]

In [16]:
# classifier = nltk.NaiveBayesClassifier.train(train_set)

full_classifier = nltk.NaiveBayesClassifier.train(featuresets)

In [17]:
print (nltk.classify.accuracy(full_classifier, test_set))

0.93


### test

In [18]:
df1 = pd.read_csv('s_med.csv')

In [19]:
sent_a = list(df1['med_sentence'].values)

In [20]:
res_s_dic = []
# res_s_dic1 = []

df_res_s = pd.DataFrame([])

sid = SentimentIntensityAnalyzer()

for i, sentence in enumerate(sent_a[:5000]):
    test_set = document_features(tknzr.tokenize(sentence))  #TODO: change to tokenize
    # res_s_dic1.append(ensclassifier.classify(test_set))
    
    prob_result = full_classifier.prob_classify(test_set)
    
    #     dic = {}
    #     dic["s"] = sentence
    #     dic["neg"] = prob_result.prob("neg")
    #     dic["pos"] = prob_result.prob("pos")
    
    # res_s_dic.append(dic)
    df_res_s.loc[i,'neg_prob'] = prob_result.prob("neg")
    df_res_s.loc[i,'pos_prob'] = prob_result.prob("pos")
    
    sid_s = sid.polarity_scores(sentence)
    df_res_s.loc[i,'neg_vader'] = sid_s['neg']
    df_res_s.loc[i,'neu_vader'] = sid_s['neu']
    df_res_s.loc[i,'pos_vader'] = sid_s['pos']
    df_res_s.loc[i,'compound'] = sid_s['compound']
    
    df_res_s.loc[i,'sentence'] = sentence
    
# df_res_s[df_res_s['pos_prob'] > .001]['sentence']
print(df_res_s['pos_prob'].max())
print(df_res_s['pos_vader'].max())
print (df_res_s)

0.885723724508
0.647
      neg_prob  pos_prob  neg_vader  neu_vader  pos_vader  compound  \
0     0.999358  0.000642      0.227      0.724      0.049   -0.7506   
1     0.999837  0.000163      0.081      0.791      0.128    0.2960   
2     0.999824  0.000176      0.000      0.851      0.149    0.5927   
3     0.999947  0.000053      0.000      1.000      0.000    0.0000   
4     0.996569  0.003431      0.032      0.770      0.198    0.8534   
5     0.999981  0.000019      0.000      1.000      0.000    0.0000   
6     0.999828  0.000172      0.000      0.695      0.305    0.6908   
7     0.999943  0.000057      0.000      0.794      0.206    0.4404   
8     0.999760  0.000240      0.083      0.917      0.000   -0.0900   
9     0.999171  0.000829      0.251      0.749      0.000   -0.7717   
10    0.999660  0.000340      0.000      1.000      0.000    0.0000   
11    0.999869  0.000131      0.052      0.862      0.086    0.2732   
12    0.998438  0.001562      0.187      0.750      0.06

In [25]:
prob_result = full_classifier.prob_classify(document_features(tknzr.tokenize(str('I love it! This is great!!!'))))
print(prob_result.prob("neg"), prob_result.prob("pos"))


0.9999248550133579 7.51449866528843e-05


In [26]:
def classfy(n):
    if n >= 0.5:
        return 'positive'
    if n <= -0.5:
        return 'negative'
    else:
        return 'neutral'

In [27]:
df_res_s['classify'] = df_res_s['compound'].map(lambda x: classfy(x))

In [28]:
df_res_s

,neg_prob,pos_prob,neg_vader,neu_vader,pos_vader,compound,sentence,classify
0,0.999358,0.000642,0.227,0.724,0.049,-0.7506,I have a feeling that this will only get worse...,negative
1,0.999837,0.000163,0.081,0.791,0.128,0.2960,I've tried medications but they only made me f...,neutral
2,0.999824,0.000176,0.000,0.851,0.149,0.5927,I'm against taking medication after what I had...,positive
3,0.999947,0.000053,0.000,1.000,0.000,0.0000,I see my doctor next week and I don't know wha...,neutral
4,0.996569,0.003431,0.032,0.770,0.198,0.8534,"A bit of a strange question, I suppose, but he...",positive
5,0.999981,0.000019,0.000,1.000,0.000,0.0000,I'm currently getting off lexapro and I've had...,neutral
6,0.999828,0.000172,0.000,0.695,0.305,0.6908,I honestly think the only thing left that woul...,positive
7,0.999943,0.000057,0.000,0.794,0.206,0.4404,How can I make it clear to whoever can actuall...,neutral
8,0.999760,0.000240,0.083,0.917,0.000,-0.0900,I've always what I'd consider moderate anxiety...,neutral
9,0.999171,0.000829,0.251,0.749,0.000,-0.7717,I scheduled a GP appointment last week and he ...,negative


In [31]:
df_res_s[['sentence','classify']].groupby('classify').count()

,sentence
classify,
negative,960
neutral,3511
positive,529


In [ ]:
df_res_s.to_csv('s_med_with_score_first5000_classify.csv')